> **Question 1. **
>
![alt text](question_ss\01.jpg "question")

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import split, explode, col, coalesce, when

In [2]:
spark = SparkSession.builder.appName("seperate_hobbies").getOrCreate()

In [3]:
data = [('Alice', 'Badminton, Tennis'), ('Bob', 'Tennis, Cricket'), ('Julis', 'Cricket, Carrom')]
columns = ["Name", "Hobbies"]
df = spark.createDataFrame(data, columns)

In [4]:
df.show()


+-----+-----------------+
| Name|          Hobbies|
+-----+-----------------+
|Alice|Badminton, Tennis|
|  Bob|  Tennis, Cricket|
|Julis|  Cricket, Carrom|
+-----+-----------------+



In [5]:
import os
print("PYSPARK_PYTHON:", os.environ.get("PYSPARK_PYTHON"))
print("PYSPARK_DRIVER_PYTHON:", os.environ.get("PYSPARK_DRIVER_PYTHON"))


PYSPARK_PYTHON: C:\Users\Prasad\AppData\Local\Programs\Python\Python311\python.exe
PYSPARK_DRIVER_PYTHON: C:\Users\Prasad\AppData\Local\Programs\Python\Python311\python.exe


In [6]:
# split by comma which convert Hobbies from str -> Array 
df.select( col("Name"), split(col("Hobbies"), ",").alias("Hobbies")).show()


+-----+--------------------+
| Name|             Hobbies|
+-----+--------------------+
|Alice|[Badminton,  Tennis]|
|  Bob|  [Tennis,  Cricket]|
|Julis|  [Cricket,  Carrom]|
+-----+--------------------+



In [7]:
# explode function convert array to multiple rows
df.select( col("Name"), explode(split(col("Hobbies"), ",")).alias("Hobbies")).show()

+-----+---------+
| Name|  Hobbies|
+-----+---------+
|Alice|Badminton|
|Alice|   Tennis|
|  Bob|   Tennis|
|  Bob|  Cricket|
|Julis|  Cricket|
|Julis|   Carrom|
+-----+---------+



![alt text](question_ss\02.jpg "question")

In [8]:
data = [('Goa', '', 'Mumbai'), ('', 'Mumbai', None), (None, '', 'Pune')]
columns = ['city1', 'city2', 'city3']
df = spark.createDataFrame(data, columns)
df.show()

+-----+------+------+
|city1| city2| city3|
+-----+------+------+
|  Goa|      |Mumbai|
|     |Mumbai|  NULL|
| NULL|      |  Pune|
+-----+------+------+



In [9]:
# This creates a new column firstnotnull that contains the first non-empty value among city1, city2, and city3 for each row.
# coalesce() Returns the first column that is not null.
df1 = df.withColumn(
    'firstnotnull', 
    coalesce(
        when(df['city1']=='', None).otherwise(df['city1']), 
        when(df['city2']=='', None).otherwise(df['city2']),
        when(df['city3']=='', None).otherwise(df['city3'])
    )
)
df1.select('firstnotnull').show()
df.show()

+------------+
|firstnotnull|
+------------+
|         Goa|
|      Mumbai|
|        Pune|
+------------+

+-----+------+------+
|city1| city2| city3|
+-----+------+------+
|  Goa|      |Mumbai|
|     |Mumbai|  NULL|
| NULL|      |  Pune|
+-----+------+------+



> **3. Claculate the % Marks for each student. Each student subject is of 100 marks. Create a result by following the below condition.**
>
![alt text](question_ss\03.jpg "question")

> Answer
>
![alt text](question_ss\03a.jpg "question")

In [10]:
data1 = [(1, "Steve"), (2, 'David'), (3, 'John'), (4, 'Shree'), (5, 'Helen')]
data2 = [(1, 'SQL', 40), (1, 'PySpark', 100), (2, 'SQL', 70), (2, 'PySpark', 60), (3, 'SQL', 30), (3, 'PySpark', 20), (4, 'SQL', 50), (4, 'PySpark', 50), (5, 'SQL', 45), (5, 'PySpark', 45)]

schema1 = ['Id', 'Name']
schema2 = ['Id', 'Subject', 'Mark']

df1 = spark.createDataFrame(data1, schema1)
df2 = spark.createDataFrame(data2, schema2)

df1.show()
df2.show()


+---+-----+
| Id| Name|
+---+-----+
|  1|Steve|
|  2|David|
|  3| John|
|  4|Shree|
|  5|Helen|
+---+-----+

+---+-------+----+
| Id|Subject|Mark|
+---+-------+----+
|  1|    SQL|  40|
|  1|PySpark| 100|
|  2|    SQL|  70|
|  2|PySpark|  60|
|  3|    SQL|  30|
|  3|PySpark|  20|
|  4|    SQL|  50|
|  4|PySpark|  50|
|  5|    SQL|  45|
|  5|PySpark|  45|
+---+-------+----+



In [11]:
# Step 1: combine both tables or dataframes by using join
df_join = df1.join(df2, df1['Id']==df2['Id']).drop(df2['Id'])
df_join.show()

+---+-----+-------+----+
| Id| Name|Subject|Mark|
+---+-----+-------+----+
|  1|Steve|    SQL|  40|
|  1|Steve|PySpark| 100|
|  2|David|    SQL|  70|
|  2|David|PySpark|  60|
|  3| John|    SQL|  30|
|  3| John|PySpark|  20|
|  4|Shree|    SQL|  50|
|  4|Shree|PySpark|  50|
|  5|Helen|    SQL|  45|
|  5|Helen|PySpark|  45|
+---+-----+-------+----+



In [12]:
from pyspark.sql.functions import sum, col, count

In [13]:
# Step 2: Calculating the percentage by groupby() on Id and Name column
#  then sum of marks divided by total count
df_per = df_join.groupBy('Id', 'Name').agg((sum(col('Mark'))/count('*')).alias('Percentage'))
df_per.show()

+---+-----+----------+
| Id| Name|Percentage|
+---+-----+----------+
|  1|Steve|      70.0|
|  2|David|      65.0|
|  3| John|      25.0|
|  4|Shree|      50.0|
|  5|Helen|      45.0|
+---+-----+----------+



In [14]:
# Step 3: use when() and otherwise() to get Result
result = df_per.select(
    '*', 
    (
        when(df_per['Percentage'] >=70, 'Distinction')
        .when((df_per['Percentage'] < 70) & (df_per['Percentage'] >= 60), 'First Class')
        .when((df_per['Percentage'] < 60) & (df_per['Percentage'] >= 50), 'Second Class')
        .when((df_per['Percentage'] < 50) & (df_per['Percentage'] >= 40), 'Third Class')
        .when(df_per['Percentage'] < 40, 'Fail')
    ).alias('Result')
)
result.show()

+---+-----+----------+------------+
| Id| Name|Percentage|      Result|
+---+-----+----------+------------+
|  1|Steve|      70.0| Distinction|
|  2|David|      65.0| First Class|
|  3| John|      25.0|        Fail|
|  4|Shree|      50.0|Second Class|
|  5|Helen|      45.0| Third Class|
+---+-----+----------+------------+



> **3. Department wise nth highest salary employees.**
>
![alt text](question_ss\04.jpg "question")

> Answer
>
![alt text](question_ss\04a.jpg "question")

In [15]:
data1=[(1,"A",1000,"IT"),(2,"B",1500,"IT"),(3,"C",2500,"IT"),(4,"D",3000,"HR"),(5,"E",2000,"HR"),(6,"F",1000,"HR")
       ,(7,"G",4000,"Sales"),(8,"H",4000,"Sales"),(9,"I",1000,"Sales"),(10,"J",2000,"Sales")]
schema1=["EmpId","EmpName","Salary","DeptName"]
df=spark.createDataFrame(data1,schema1)
df.show()

+-----+-------+------+--------+
|EmpId|EmpName|Salary|DeptName|
+-----+-------+------+--------+
|    1|      A|  1000|      IT|
|    2|      B|  1500|      IT|
|    3|      C|  2500|      IT|
|    4|      D|  3000|      HR|
|    5|      E|  2000|      HR|
|    6|      F|  1000|      HR|
|    7|      G|  4000|   Sales|
|    8|      H|  4000|   Sales|
|    9|      I|  1000|   Sales|
|   10|      J|  2000|   Sales|
+-----+-------+------+--------+



In [16]:
from pyspark.sql.functions import *
from pyspark.sql.window import *

In [17]:
df_rank = df.select(
    '*',
    dense_rank().over(
        Window.partitionBy(df['DeptName'])
        .orderBy(df['Salary'].desc())
    ).alias('rank')
)

df_rank.show()

+-----+-------+------+--------+----+
|EmpId|EmpName|Salary|DeptName|rank|
+-----+-------+------+--------+----+
|    4|      D|  3000|      HR|   1|
|    5|      E|  2000|      HR|   2|
|    6|      F|  1000|      HR|   3|
|    3|      C|  2500|      IT|   1|
|    2|      B|  1500|      IT|   2|
|    1|      A|  1000|      IT|   3|
|    7|      G|  4000|   Sales|   1|
|    8|      H|  4000|   Sales|   1|
|   10|      J|  2000|   Sales|   2|
|    9|      I|  1000|   Sales|   3|
+-----+-------+------+--------+----+



In [18]:
res = df_rank.filter(df_rank.rank==1)
res.show()

+-----+-------+------+--------+----+
|EmpId|EmpName|Salary|DeptName|rank|
+-----+-------+------+--------+----+
|    4|      D|  3000|      HR|   1|
|    3|      C|  2500|      IT|   1|
|    7|      G|  4000|   Sales|   1|
|    8|      H|  4000|   Sales|   1|
+-----+-------+------+--------+----+



> **5.**
>
![alt text](question_ss\05.jpg "question")

> Answer
>
![alt text](question_ss\05a.jpg "question")

In [19]:
# Employees Salary info
data1=[(100,"Raj",None,1,"01-04-23",50000), (200,"Joanne",100,1,"01-04-23",4000),
       (200,"Joanne",100,1,"13-04-23",4500),(200,"Joanne",100,1,"14-04-23",4020)]

schema1=["EmpId","EmpName","Mgrid","deptid","salarydt","salary"]

df_salary=spark.createDataFrame(data1,schema1)
df_salary.show()

#department dataframe
data2=[(1,"IT"), (2,"HR")]
schema2=["deptid","deptname"]

df_dept=spark.createDataFrame(data2,schema2)
df_dept.show()

+-----+-------+-----+------+--------+------+
|EmpId|EmpName|Mgrid|deptid|salarydt|salary|
+-----+-------+-----+------+--------+------+
|  100|    Raj| NULL|     1|01-04-23| 50000|
|  200| Joanne|  100|     1|01-04-23|  4000|
|  200| Joanne|  100|     1|13-04-23|  4500|
|  200| Joanne|  100|     1|14-04-23|  4020|
+-----+-------+-----+------+--------+------+

+------+--------+
|deptid|deptname|
+------+--------+
|     1|      IT|
|     2|      HR|
+------+--------+



In [20]:
# reformat date column 
df = df_salary.withColumn('Newsaldt', to_date(col('salarydt'), 'dd-MM-yy'))
df.show()

+-----+-------+-----+------+--------+------+----------+
|EmpId|EmpName|Mgrid|deptid|salarydt|salary|  Newsaldt|
+-----+-------+-----+------+--------+------+----------+
|  100|    Raj| NULL|     1|01-04-23| 50000|2023-04-01|
|  200| Joanne|  100|     1|01-04-23|  4000|2023-04-01|
|  200| Joanne|  100|     1|13-04-23|  4500|2023-04-13|
|  200| Joanne|  100|     1|14-04-23|  4020|2023-04-14|
+-----+-------+-----+------+--------+------+----------+



In [21]:
# join both df, join with only ['deptid] so that it will remove duplicate deptid col
df_join = df.join(df_dept, ['deptid'])

# Self join to get manager details
df_joined = df_join.alias('tbl1').join(
	df_join.alias('tbl2'),
	col('tbl1.Mgrid') == col('tbl2.EmpId'),
	'left'
).select(
    col('tbl1.deptname'),
	col('tbl2.EmpName').alias('ManagerName'),
	col('tbl1.EmpName').alias('EmpName'),
	col('tbl1.Newsaldt').alias('Newsaldt'),
	col('tbl1.salary').alias('salary'),
)

df_joined.show()

+--------+-----------+-------+----------+------+
|deptname|ManagerName|EmpName|  Newsaldt|salary|
+--------+-----------+-------+----------+------+
|      IT|       NULL|    Raj|2023-04-01| 50000|
|      IT|        Raj| Joanne|2023-04-01|  4000|
|      IT|        Raj| Joanne|2023-04-13|  4500|
|      IT|        Raj| Joanne|2023-04-14|  4020|
+--------+-----------+-------+----------+------+



In [22]:
# groupby
res = df_joined.groupBy('deptname', 'ManagerName', 'EmpName', year('Newsaldt').alias('Year'), date_format('Newsaldt', 'MMM').alias('Month')).sum('salary')
res.show()

+--------+-----------+-------+----+-----+-----------+
|deptname|ManagerName|EmpName|Year|Month|sum(salary)|
+--------+-----------+-------+----+-----+-----------+
|      IT|        Raj| Joanne|2023|  Apr|      12520|
|      IT|       NULL|    Raj|2023|  Apr|      50000|
+--------+-----------+-------+----+-----+-----------+



> **6. How to check data skew Issue and how to solve it.**
>
![alt text](question_ss\06.jpg "question")

> **6. Merger two dataframes**


In [25]:
simpleData = [(1, "Sagar", "CSE", "UP", 80), (2, "Shivam", "IT", "MP", 86), (3, "Muni", "Mech", "AP", 70)]

simpleData_2 = [(5, "Raj", "CSE", "HP"), (7, "Kunal", "Mech", "Rajasthan")]


columns_1 = ["ID", "Student_Name", "Department_Name", "City", "Marks"]
columns_2 = ["ID", "Student_Name", "Department_Name", "City"]

df_1 = spark.createDataFrame(data = simpleData, schema = columns_1)
df_2 = spark.createDataFrame(data = simpleData_2, schema = columns_2)
df_1.show()
df_2.show()

+---+------------+---------------+----+-----+
| ID|Student_Name|Department_Name|City|Marks|
+---+------------+---------------+----+-----+
|  1|       Sagar|            CSE|  UP|   80|
|  2|      Shivam|             IT|  MP|   86|
|  3|        Muni|           Mech|  AP|   70|
+---+------------+---------------+----+-----+

+---+------------+---------------+---------+
| ID|Student_Name|Department_Name|     City|
+---+------------+---------------+---------+
|  5|         Raj|            CSE|       HP|
|  7|       Kunal|           Mech|Rajasthan|
+---+------------+---------------+---------+



In [27]:
df = df_1.unionByName(df_2, allowMissingColumns=True)
df.show()

+---+------------+---------------+---------+-----+
| ID|Student_Name|Department_Name|     City|Marks|
+---+------------+---------------+---------+-----+
|  1|       Sagar|            CSE|       UP|   80|
|  2|      Shivam|             IT|       MP|   86|
|  3|        Muni|           Mech|       AP|   70|
|  5|         Raj|            CSE|       HP| NULL|
|  7|       Kunal|           Mech|Rajasthan| NULL|
+---+------------+---------------+---------+-----+

